In [28]:
import numpy as np
import pandas as pd

from sklearn.impute import SimpleImputer, KNNImputer, MissingIndicator

import warnings
warnings.filterwarnings('ignore')

In [29]:
data: dict = {
    'id' : [1000, 1001, 1002, 1003, 1004, 1005, 1006],
    'date' : ['20230101', '20230102', '20230103', '20230104', '20230105', '20230106', '20230107'],
    'age' : [21, 56, 33, 49, 27, 42, 32],
    'income' : [67000, 220000, 97000, 166000, 81000, 157000, 96000],
    'gender' : ['Male', 'Female', 'Female', 'Male', 'Male', 'Female', 'Female'],
    'education' : ['Bachelos', 'PhD', 'Masters', 'Masters', 'Bachelos', 'Bachelos', 'Bachelos'],
    'passed' : [False, True, True, True, False, False, True],
    'measurement' : np.random.randn(7).round(2),
}
df = pd.DataFrame(data)
df

,id,date,age,income,gender,education,passed,measurement
0,1000,20230101,21,67000,Male,Bachelos,False,2.36
1,1001,20230102,56,220000,Female,PhD,True,-0.53
2,1002,20230103,33,97000,Female,Masters,True,-0.55
3,1003,20230104,49,166000,Male,Masters,True,0.36
4,1004,20230105,27,81000,Male,Bachelos,False,1.67
5,1005,20230106,42,157000,Female,Bachelos,False,-0.40
6,1006,20230107,32,96000,Female,Bachelos,True,0.93


### 컬럼간의 상관계수 계산

In [30]:
df.corr()

,id,age,income,passed,measurement
id,1.000000,-0.006207,-0.076124,0.000000,-0.121012
age,-0.006207,1.000000,0.981673,0.537583,-0.757754
income,-0.076124,0.981673,1.000000,0.413131,-0.713190
passed,0.000000,0.537583,0.413131,1.000000,-0.536157
measurement,-0.121012,-0.757754,-0.713190,-0.536157,1.000000


### 컬럼정보 확인

In [31]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7 entries, 0 to 6
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   id           7 non-null      int64  
 1   date         7 non-null      object 
 2   age          7 non-null      int64  
 3   income       7 non-null      int64  
 4   gender       7 non-null      object 
 5   education    7 non-null      object 
 6   passed       7 non-null      bool   
 7   measurement  7 non-null      float64
dtypes: bool(1), float64(1), int64(3), object(3)
memory usage: 527.0+ bytes


### 데이터 값 출력

In [32]:
df.values

array([[1000, '20230101', 21, 67000, 'Male', 'Bachelos', False, 2.36],
       [1001, '20230102', 56, 220000, 'Female', 'PhD', True, -0.53],
       [1002, '20230103', 33, 97000, 'Female', 'Masters', True, -0.55],
       [1003, '20230104', 49, 166000, 'Male', 'Masters', True, 0.36],
       [1004, '20230105', 27, 81000, 'Male', 'Bachelos', False, 1.67],
       [1005, '20230106', 42, 157000, 'Female', 'Bachelos', False, -0.4],
       [1006, '20230107', 32, 96000, 'Female', 'Bachelos', True, 0.93]],
      dtype=object)

### 데이터 결측치 임의로 삽입

In [33]:
df2 = df.copy()
df2.iloc[[3,5], [1, 2, 4, 5, 6, 7]] = np.nan
df2

,id,date,age,income,gender,education,passed,measurement
0,1000,20230101,21.0,67000,Male,Bachelos,False,2.36
1,1001,20230102,56.0,220000,Female,PhD,True,-0.53
2,1002,20230103,33.0,97000,Female,Masters,True,-0.55
3,1003,NaN,NaN,166000,NaN,NaN,NaN,NaN
4,1004,20230105,27.0,81000,Male,Bachelos,False,1.67
5,1005,NaN,NaN,157000,NaN,NaN,NaN,NaN
6,1006,20230107,32.0,96000,Female,Bachelos,True,0.93


### 각 컬럼별 결측치 확인

In [34]:
df2.isnull().sum()

id             0
date           2
age            2
income         0
gender         2
education      2
passed         2
measurement    2
dtype: int64

### 결측치가 포함된 데이터 출력

In [35]:
df2[df2.isnull().any(axis=1)]

,id,date,age,income,gender,education,passed,measurement
3,1003,NaN,NaN,166000,NaN,NaN,NaN,NaN
5,1005,NaN,NaN,157000,NaN,NaN,NaN,NaN


### 결측치가 포함된 행 삭제

In [36]:
df2.dropna(axis=0)

,id,date,age,income,gender,education,passed,measurement
0,1000,20230101,21.0,67000,Male,Bachelos,False,2.36
1,1001,20230102,56.0,220000,Female,PhD,True,-0.53
2,1002,20230103,33.0,97000,Female,Masters,True,-0.55
4,1004,20230105,27.0,81000,Male,Bachelos,False,1.67
6,1006,20230107,32.0,96000,Female,Bachelos,True,0.93


### 결측치가 포함된 열 삭제

In [37]:
df2.dropna(axis=1)


,id,income
0,1000,67000
1,1001,220000
2,1002,97000
3,1003,166000
4,1004,81000
5,1005,157000
6,1006,96000


### 결측치를 특정 값으로 치환

In [38]:
df2.loc[:,['age','measurement']].fillna(-9999)

,age,measurement
0,21.0,2.36
1,56.0,-0.53
2,33.0,-0.55
3,-9999.0,-9999.00
4,27.0,1.67
5,-9999.0,-9999.00
6,32.0,0.93


In [39]:
df2.loc[:,['age','measurement']].fillna(df2.mean()[['age','measurement']])


,age,measurement
0,21.0,2.360
1,56.0,-0.530
2,33.0,-0.550
3,33.8,0.776
4,27.0,1.670
5,33.8,0.776
6,32.0,0.930


### sklearn 패키지를 활용하여 결측치 처리
- impute를 사용하여 결측치 처리(특정값)

In [40]:
imp = SimpleImputer(missing_values=np.nan, strategy='constant', fill_value=-9999)
imputed = imp.fit_transform(df2.loc[:, ['age','measurement']].values)
pd.DataFrame(imputed, columns=['age','measurement'])

,age,measurement
0,21.0,2.36
1,56.0,-0.53
2,33.0,-0.55
3,-9999.0,-9999.00
4,27.0,1.67
5,-9999.0,-9999.00
6,32.0,0.93



- impute를 사용하여 결측치 처리(평균)

In [41]:
imp = SimpleImputer(missing_values=np.nan, strategy='mean')
imputed = imp.fit_transform(df2.loc[:, ['age','measurement']].values)
pd.DataFrame(imputed, columns=['age','measurement'])

,age,measurement
0,21.0,2.360
1,56.0,-0.530
2,33.0,-0.550
3,33.8,0.776
4,27.0,1.670
5,33.8,0.776
6,32.0,0.930


### imputer를 활용하여 결측치 처리(n_neighbor)

In [42]:
imp_knn = KNNImputer(n_neighbors=2, weights='uniform')
imputed = imp_knn.fit_transform(df2.loc[:, ['age','measurement']].values)
pd.DataFrame(imputed, columns=['age','measurement'])

,age,measurement
0,21.0,2.360
1,56.0,-0.530
2,33.0,-0.550
3,33.8,0.776
4,27.0,1.670
5,33.8,0.776
6,32.0,0.930


### 문자열에 대한 결측치 처리
- 특정값으로 치환

In [43]:
df2.loc[:,['gender','education']].fillna('Unknown')


,gender,education
0,Male,Bachelos
1,Female,PhD
2,Female,Masters
3,Unknown,Unknown
4,Male,Bachelos
5,Unknown,Unknown
6,Female,Bachelos


- mode(최빈값)을 활용하여 치환

In [44]:
df2.loc[:,['gender','education']].fillna(df2.mode().iloc[0])


,gender,education
0,Male,Bachelos
1,Female,PhD
2,Female,Masters
3,Female,Bachelos
4,Male,Bachelos
5,Female,Bachelos
6,Female,Bachelos


### sklearn을 활용하여 문자열 결측치 치환

In [45]:
imp = SimpleImputer(missing_values=np.nan, strategy='constant', fill_value='Unknown')
imputed = imp.fit_transform(df2.loc[:, ['gender','education']].values)
pd.DataFrame(imputed, columns=['gender','education'])

,gender,education
0,Male,Bachelos
1,Female,PhD
2,Female,Masters
3,Unknown,Unknown
4,Male,Bachelos
5,Unknown,Unknown
6,Female,Bachelos


In [46]:
imp = SimpleImputer(missing_values=np.nan, strategy='most_frequent')
imputed = imp.fit_transform(df2.loc[:,['gender', 'education']].values)
pd.DataFrame(imputed,columns=['gender','education'])

,gender,education
0,Male,Bachelos
1,Female,PhD
2,Female,Masters
3,Female,Bachelos
4,Male,Bachelos
5,Female,Bachelos
6,Female,Bachelos


### 문자열 타입을 DateTime 타입으로 type casting 하기

In [47]:
df2.date

0    20230101
1    20230102
2    20230103
3         NaN
4    20230105
5         NaN
6    20230107
Name: date, dtype: object

In [48]:
df2.date.dtype # date의 타입을 확인 가능

dtype('O')

In [49]:
df2.date = pd.to_datetime(df2.date) # object 타입을 datetime 타입으로 변경
df2.date.dtype # object 타입을 확인 가능


dtype('<M8[ns]')

In [54]:
df2.date[1].dayofweek

0

In [52]:
df2.date[1].weekofyear

1

In [55]:
df2['date'] = pd.date_range(start=df2.date.min(),end=df2.date.max(), freq='D')
df2['dayofweek'] = df2['date'].apply(lambda x: x.dayofweek)
df2


,id,date,age,income,gender,education,passed,measurement,dayofweek
0,1000,2023-01-01,21.0,67000,Male,Bachelos,False,2.36,6
1,1001,2023-01-02,56.0,220000,Female,PhD,True,-0.53,0
2,1002,2023-01-03,33.0,97000,Female,Masters,True,-0.55,1
3,1003,2023-01-04,NaN,166000,NaN,NaN,NaN,NaN,2
4,1004,2023-01-05,27.0,81000,Male,Bachelos,False,1.67,3
5,1005,2023-01-06,NaN,157000,NaN,NaN,NaN,NaN,4
6,1006,2023-01-07,32.0,96000,Female,Bachelos,True,0.93,5
